# Initial Settings

In [1]:
%%capture
!pip install openai==0.28

In [2]:
# LLM
import openai

# Data Processing
import pandas as pd
import re

# settings
from google.colab import userdata, drive
from tqdm import tqdm
import os

In [3]:
# finding folder path
def find_folder(root, target):
  for root, dirs, _ in os.walk(root):
    if target in dirs:
      return os.path.join(root, target)
  return ValueError("None\nDownload the Dataset or change the target Directory")

In [4]:
drive.mount("/content/drive")

root = "/"
target = "datasets_imdb"
PATH = find_folder(root, target)

Mounted at /content/drive


# Testing Prompts

I will test 4 prompts generated by ChatGPT with 15 random users. All of them will be evaluated, so the better one will be used in large scale.

## LLM

In [5]:
openai.api_key = userdata.get('OPENAI_API_KEY')

def request_openai(prompt, openai=openai):

  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      temperature=0.2,
      max_tokens=300,
      top_p=0.9,
      messages=[
          {
              "role": "system",
              "content": "You are a recommender"
          },
          {
              "role": "user",
              "content": prompt
          }
      ]
  )

  return response['choices'][0]['message']['content']

## Reading Data

In [5]:
headers = ["userId", "movieId", "rating", "timestamp", "title"]
trainset = pd.read_csv(f"{PATH}/refined/train_llm.csv", names=headers)
testset = pd.read_csv(f"{PATH}/refined/test_llm.csv", names=headers)

In [6]:
users_test = trainset.userId.unique()[:15]
trainset = trainset[trainset.userId.isin(users_test)]

In [7]:
user_movies_dict = trainset.groupby('userId')['title'].apply(list).to_dict()

## Prompts

In [8]:
testing_prompts = {
  "p1": "Please suggest 15 movies or TV shows. Format the output as follows: 'production (release year)\nproduction (release year)\nproduction (release year)...'",
  "p2": "I'm looking for movies and TV shows, can you provide 15 recommendations released prior to 2015? Please list 15 productions with their release years only in the format: 'production (release year)\nproduction (release year)...'",
  "p3": "Could you recommend some classic movies and TV shows from before 2015? I need the output formatted as follows: 'movie (release year)\ntv show (release year)\nmovie (release year)...'",
  "p4": "I'm interested in discovering older content. Can you suggest 15 movies or TV shows released before 2015? Please list the productions along with their release years only, in the format: 'production (release year)\nproduction (release year)...'"
}

## Requesting Recs

In [26]:
llm_recs = list()

total_iterations = len(testing_prompts) * len(user_movies_dict)
progress_bar = tqdm(total=total_iterations, desc='Progress')

for prompt_id, prompt in testing_prompts.items():
  for k, v in user_movies_dict.items():
    temp_dict = dict()

    movies = "\n".join(v)
    prompt += f"\nI've watched:\n{movies}"
    response = request_openai(prompt)

    temp_dict['prompt_id'] = prompt_id
    temp_dict["response"] = response
    temp_dict["userId"] = k

    llm_recs.append(temp_dict)
    progress_bar.update(1)

progress_bar.close()

Progress: 100%|██████████| 60/60 [03:15<00:00,  3.26s/it]


In [28]:
df_recs = pd.DataFrame(llm_recs)
df_recs.tail(5)

,prompt_id,response,userId
55,p4,The recommendations are:\n1. The Shawshank Red...,2854
56,p4,The recommendations are:\n1. The Shawshank Red...,3231
57,p4,The recommendations are:\n1. The Godfather (19...,3300
58,p4,1. The Shawshank Redemption (1994)\n2. Pulp Fi...,7623
59,p4,Here are 15 movies or TV shows released before...,7886


# Saving Recs

In [10]:
df_recs.to_csv(f"{PATH}/refined/recs_llm/evaluating_prompts.csv", index=False)

# Evaluating Better Prompt

## Visual Analysis

In [9]:
df_recs = pd.read_csv(f"{PATH}/refined/recs_llm/evaluating_prompts.csv")

for k in testing_prompts.keys():
  p_recs = df_recs[df_recs.prompt_id == k]['response']
  print(f"\n\n{'#'*10} {k} {'#'*10}\n\n")
  for x in df_recs[df_recs.prompt_id == 'p1']['response']:
    print(x)



########## p1 ##########


The Shawshank Redemption (1994)
Friends (1994)
Pulp Fiction (1994)
The Lion King (1994)
Forrest Gump (1994)
The Matrix (1999)
The Sopranos (1999)
American Beauty (1999)
Fight Club (1999)
The Sixth Sense (1999)
The Green Mile (1999)
The Office (UK) (2001)
The Lord of the Rings: The Fellowship of the Ring (2001)
Harry Potter and the Sorcerer's Stone (2001)
Shrek (2001)
The Princess Bride (1987)
The Shawshank Redemption (1994)
Forrest Gump (1994)
Pulp Fiction (1994)
The Matrix (1999)
The Silence of the Lambs (1991)
Fight Club (1999)
The Godfather (1972)
Back to the Future (1985)
Jurassic Park (1993)
Friends (1994)
The X-Files (1993)
The Simpsons (1989)
Seinfeld (1989)
Buffy the Vampire Slayer (1997)
The Sixth Sense (1999)
The Matrix (1999)
American Beauty (1999)
Fight Club (1999)
The Green Mile (1999)
The Shawshank Redemption (1994)
Pulp Fiction (1994)
Forrest Gump (1994)
The Lion King (1994)
Jurassic Park (1993)
Friends (1994)
The X-Files (1993)
The Simpsons 

In [10]:
for index, row in df_recs.iterrows():
  movie_name, prompt = row['response'], row['prompt_id']
  print(f"#### {prompt} #### \n{movie_name}\n\n")

#### p1 #### 
The Shawshank Redemption (1994)
Friends (1994)
Pulp Fiction (1994)
The Lion King (1994)
Forrest Gump (1994)
The Matrix (1999)
The Sopranos (1999)
American Beauty (1999)
Fight Club (1999)
The Sixth Sense (1999)
The Green Mile (1999)
The Office (UK) (2001)
The Lord of the Rings: The Fellowship of the Ring (2001)
Harry Potter and the Sorcerer's Stone (2001)
Shrek (2001)


#### p1 #### 
The Princess Bride (1987)
The Shawshank Redemption (1994)
Forrest Gump (1994)
Pulp Fiction (1994)
The Matrix (1999)
The Silence of the Lambs (1991)
Fight Club (1999)
The Godfather (1972)
Back to the Future (1985)
Jurassic Park (1993)
Friends (1994)
The X-Files (1993)
The Simpsons (1989)
Seinfeld (1989)
Buffy the Vampire Slayer (1997)


#### p1 #### 
The Sixth Sense (1999)
The Matrix (1999)
American Beauty (1999)
Fight Club (1999)
The Green Mile (1999)
The Shawshank Redemption (1994)
Pulp Fiction (1994)
Forrest Gump (1994)
The Lion King (1994)
Jurassic Park (1993)
Friends (1994)
The X-Files (19

I'll disqualify p3 and p4.

- p4 output does not have a clear pattern: headers can be included or not. It'll be harder to adjust the returns.

- p3 did not respected the amount of recs required.

## MAP

### Reading all movies names

In [15]:
movies_names = pd.read_csv(f"{PATH}/raw/movielens_20m_datasets/movie.csv")
movies_names.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Functions

In [11]:
def item_is_relevant(user_id, item_id, df):
    aux = df[df["userId"] == user_id]
    if item_id in list(aux['title']):
        return True
    return False

In [12]:
def calculate_MAP(prediction_user_map, movies):

  progress_bar = tqdm(total=len(prediction_user_map), desc='Progress', position=0)
  MAP = 0

  for user_id in prediction_user_map:
    map_user, user_relevants = 0, 0
    for index, item in enumerate(prediction_user_map[user_id]):
      relevance = item_is_relevant(user_id, item, movies)
      user_relevants += int(relevance)
      map_user += user_relevants / (index + 1) if relevance else 0

    MAP += map_user / user_relevants if user_relevants != 0 else 0

    progress_bar.update(1)
  progress_bar.close()

  return round(MAP / len(prediction_user_map), 3)

In [31]:
def fix_response(response: str) -> list:
  # if it's an indexed list, I'll split and remove the integers
  if "." in response:
    movies = re.findall(r'\d+\.\s+(.+)', response)
    return movies
  else:
    pattern = r'((?:The\s)?(?:\b[A-Z][a-z]*\s?)+)\s\((\d{4})\)'
    movies = re.findall(pattern, response)
    movies = [f"{item} ({year})" for item, year in movies]

    return movies

In [40]:
# if the movie is not in IMDB dataset, the rec will be dropped
def in_dataset(user_interactions: str, all_movies):
  movies_in_dataset = [movie for movie in user_interactions if movie in list(all_movies.title)]

  return movies_in_dataset[:10]

### Eval - prompt 1

In [19]:
df_recs_a = df_recs[df_recs["prompt_id"] == "p1"]
df_recs_a.head(5)

,prompt_id,response,userId
0,p1,The Shawshank Redemption (1994)\nFriends (1994...,810
1,p1,The Princess Bride (1987)\nThe Shawshank Redem...,897
2,p1,The Sixth Sense (1999)\nThe Matrix (1999)\nAme...,1302
3,p1,The Godfather (1972)\nFriends (1994)\nForrest ...,1738
4,p1,The Godfather (1972)\nFriends (1994)\nThe Shaw...,1932


In [43]:
prediction_user_map = dict()

for index, row in df_recs_a.iterrows():
  movies = row['response']
  movies = fix_response(movies)
  movies = in_dataset(movies, movies_names)

  prediction_user_map[row["userId"]] = movies

In [44]:
prediction_user_map

{810: ['Pulp Fiction (1994)',
  'Forrest Gump (1994)',
  'American Beauty (1999)',
  'Fight Club (1999)',
  'Shrek (2001)'],
 897: ['Forrest Gump (1994)',
  'Pulp Fiction (1994)',
  'Fight Club (1999)',
  'Jurassic Park (1993)'],
 1302: ['American Beauty (1999)',
  'Fight Club (1999)',
  'Pulp Fiction (1994)',
  'Forrest Gump (1994)',
  'Jurassic Park (1993)'],
 1738: ['Forrest Gump (1994)',
  'Pulp Fiction (1994)',
  'Jurassic Park (1993)',
  'Titanic (1997)',
  'Braveheart (1995)',
  'Home Alone (1990)'],
 1932: ['Forrest Gump (1994)',
  'Pulp Fiction (1994)',
  'Jurassic Park (1993)',
  'Titanic (1997)'],
 1971: ['Forrest Gump (1994)',
  'Jurassic Park (1993)',
  'Home Alone (1990)',
  'Pulp Fiction (1994)'],
 2116: ['Forrest Gump (1994)',
  'Pulp Fiction (1994)',
  'Jurassic Park (1993)',
  'Goodfellas (1990)'],
 2162: ['Pulp Fiction (1994)',
  'Forrest Gump (1994)',
  'Jurassic Park (1993)',
  'Titanic (1997)'],
 2577: ['Casablanca (1942)',
  'American Pie (1999)',
  'American Bea

In [45]:
map_p1 = calculate_MAP(prediction_user_map, testset)

Progress: 100%|██████████| 15/15 [00:00<00:00, 459.95it/s]


## Eval - prompt 2

In [49]:
df_recs_b = df_recs[df_recs["prompt_id"] == "p2"]
df_recs_a.head(5)

,prompt_id,response,userId
15,p2,1. The Matrix (1999)\n2. Fight Club (1999)\n3....,810
16,p2,The Shawshank Redemption (1994)\nPulp Fiction ...,897
17,p2,The recommendations are:\n\n1. The Shawshank R...,1302
18,p2,The Sixth Sense (1999)\nThe Matrix (1999)\nFig...,1738
19,p2,The Matrix (1999)\nThe Shawshank Redemption (1...,1932


In [50]:
prediction_user_map = dict()

for index, row in df_recs_b.iterrows():
  movies = row['response']
  movies = fix_response(movies)
  movies = in_dataset(movies, movies_names)

  prediction_user_map[row["userId"]] = movies

In [51]:
map_p2 = calculate_MAP(prediction_user_map, testset)

Progress: 100%|██████████| 15/15 [00:00<00:00, 411.60it/s]


## Results

In [52]:
print(f"MAP (prompt 1) = {map_p1}\nMAP (prompt 2) = {map_p2}")

MAP (prompt 1) = 0.397
MAP (prompt 2) = 0.525
